### 다중회귀분석을 위한 데이터 전처리 작업
#### 1. 소방청 데이터(2019년)의 법정동 데이터를 행정동으로 변환
#### 2. 날짜 데이터의 시:분 --> 시의 형태로 변환

In [1]:
import pandas as pd
import numpy as np

In [49]:
data = pd.read_csv("data/소방청_연간화재통계_20191231.csv", encoding='ansi')
display(data.head())

,화재발생년월일,시도,시·군·구,읍면동,화재유형,발화열원대분류,발화열원소분류,발화요인대분류,발화요인소분류,최초착화물대분류,최초착화물소분류,인명피해(명)소계,사망,부상,재산피해소계,장소대분류,장소중분류,장소소분류
0,2019-01-01 00:03,강원도,삼척시,오분동,기타(쓰레기 화재등),"폭발물, 폭죽",폭죽,부주의,폭죽놀이,"종이,목재,건초등","풀, 나뭇잎",1,0,1,0,기타,야외,기타야외
1,2019-01-01 00:06,경기도,평택시,안중읍,"건축,구조물",미상,미상,미상,미상,미상,미상,0,0,0,14900,주거,단독주택,단독주택
2,2019-01-01 00:40,서울특별시,성북구,하월곡동,"자동차,철도차량",미상,미상,미상,미상,미상,미상,0,0,0,3642,"자동차,철도차량",자동차,승용자동차
3,2019-01-01 00:52,경기도,이천시,설성면,기타(쓰레기 화재등),화학적 발화열,화학반응열,화학적 요인,자연발화,"종이,목재,건초등","풀, 나뭇잎",0,0,0,1848,기타,야외,야적장
4,2019-01-01 01:06,부산광역시,부산진구,당감동,"건축,구조물",작동기기,기타(작동기기),전기적 요인,절연열화에 의한 단락,"전기,전자","전기, 전자기기 기판",0,0,0,45,기타서비스,기타건축물,기타 건축물


#### 필요한 칼럼만 잘라냄 --> 화재발생년월일, 시도, 시군구, 읍면동

In [50]:
data1 = data.loc[:, ['화재발생년월일', '시도', '시·군·구', '읍면동']]
data1.head()

,화재발생년월일,시도,시·군·구,읍면동
0,2019-01-01 00:03,강원도,삼척시,오분동
1,2019-01-01 00:06,경기도,평택시,안중읍
2,2019-01-01 00:40,서울특별시,성북구,하월곡동
3,2019-01-01 00:52,경기도,이천시,설성면
4,2019-01-01 01:06,부산광역시,부산진구,당감동


#### 날짜데이터의 형식 data1.info()로 확인
#### object --> datetime으로 변경

In [2]:
from datetime import datetime, timezone

In [52]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40103 entries, 0 to 40102
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   화재발생년월일  40103 non-null  object
 1   시도       40103 non-null  object
 2   시·군·구    40103 non-null  object
 3   읍면동      40103 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [53]:
data1['fire_time'] = data1['화재발생년월일'].astype('str')
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40103 entries, 0 to 40102
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   화재발생년월일    40103 non-null  object
 1   시도         40103 non-null  object
 2   시·군·구      40103 non-null  object
 3   읍면동        40103 non-null  object
 4   fire_time  40103 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB


In [54]:
data1[['화재시간', '분']] = pd.DataFrame(data1.fire_time.str.split(':', 2).tolist())
display(data1.head())

,화재발생년월일,시도,시·군·구,읍면동,fire_time,화재시간,분
0,2019-01-01 00:03,강원도,삼척시,오분동,2019-01-01 00:03,2019-01-01 00,03
1,2019-01-01 00:06,경기도,평택시,안중읍,2019-01-01 00:06,2019-01-01 00,06
2,2019-01-01 00:40,서울특별시,성북구,하월곡동,2019-01-01 00:40,2019-01-01 00,40
3,2019-01-01 00:52,경기도,이천시,설성면,2019-01-01 00:52,2019-01-01 00,52
4,2019-01-01 01:06,부산광역시,부산진구,당감동,2019-01-01 01:06,2019-01-01 01,06


In [55]:
data1 = data1.loc[:, ['화재시간', '시도', '시·군·구', '읍면동']]
display(data1.head())

,화재시간,시도,시·군·구,읍면동
0,2019-01-01 00,강원도,삼척시,오분동
1,2019-01-01 00,경기도,평택시,안중읍
2,2019-01-01 00,서울특별시,성북구,하월곡동
3,2019-01-01 00,경기도,이천시,설성면
4,2019-01-01 01,부산광역시,부산진구,당감동


In [56]:
data1['화재시간'] = pd.to_datetime(data1['화재시간'])
display(data1.head())

,화재시간,시도,시·군·구,읍면동
0,2019-01-01 00:00:00,강원도,삼척시,오분동
1,2019-01-01 00:00:00,경기도,평택시,안중읍
2,2019-01-01 00:00:00,서울특별시,성북구,하월곡동
3,2019-01-01 00:00:00,경기도,이천시,설성면
4,2019-01-01 01:00:00,부산광역시,부산진구,당감동


In [57]:
data1['fire_time'] = data1['화재시간'].dt.strftime('%Y-%m-%d %H:00')
display(data1.head())

,화재시간,시도,시·군·구,읍면동,fire_time
0,2019-01-01 00:00:00,강원도,삼척시,오분동,2019-01-01 00:00
1,2019-01-01 00:00:00,경기도,평택시,안중읍,2019-01-01 00:00
2,2019-01-01 00:00:00,서울특별시,성북구,하월곡동,2019-01-01 00:00
3,2019-01-01 00:00:00,경기도,이천시,설성면,2019-01-01 00:00
4,2019-01-01 01:00:00,부산광역시,부산진구,당감동,2019-01-01 01:00


In [58]:
data1 = data1.drop(data1.columns[[0]], axis=1)

In [59]:
display(data1.head())

,시도,시·군·구,읍면동,fire_time
0,강원도,삼척시,오분동,2019-01-01 00:00
1,경기도,평택시,안중읍,2019-01-01 00:00
2,서울특별시,성북구,하월곡동,2019-01-01 00:00
3,경기도,이천시,설성면,2019-01-01 00:00
4,부산광역시,부산진구,당감동,2019-01-01 01:00


In [60]:
data1['법정동주소'] = data1['시도'] + " " + data1['시·군·구'] + " " + data1['읍면동']
data1 = data1.drop(data1.columns[[0, 1, 2]], axis=1)

In [61]:
display(data1.head())

,fire_time,법정동주소
0,2019-01-01 00:00,강원도 삼척시 오분동
1,2019-01-01 00:00,경기도 평택시 안중읍
2,2019-01-01 00:00,서울특별시 성북구 하월곡동
3,2019-01-01 00:00,경기도 이천시 설성면
4,2019-01-01 01:00,부산광역시 부산진구 당감동


In [62]:
data = pd.read_excel("data/주소리스트.xlsx")

data = data.drop(data.columns[[0, 4, 6, 7]], axis=1) # 행정동코드, 법정동코드, 생성일자, 말소일자 제거
data.drop([0, 1], inplace=True) # 필요없는 행 제거
data.set_index('시도명', inplace=True)
data.reset_index(inplace=True) # 행 번호 초기화(0부터 시작)
data['행정동주소'] = data['시도명'] + " " + data['시군구명'] + " " + data['읍면동명']
data['법정동주소'] = data['시도명'] + " " + data['시군구명'] + " " + data['동리명']
display(data.head())

,시도명,시군구명,읍면동명,동리명,행정동주소,법정동주소
0,서울특별시,종로구,청운효자동,청운동,서울특별시 종로구 청운효자동,서울특별시 종로구 청운동
1,서울특별시,종로구,청운효자동,신교동,서울특별시 종로구 청운효자동,서울특별시 종로구 신교동
2,서울특별시,종로구,청운효자동,궁정동,서울특별시 종로구 청운효자동,서울특별시 종로구 궁정동
3,서울특별시,종로구,청운효자동,효자동,서울특별시 종로구 청운효자동,서울특별시 종로구 효자동
4,서울특별시,종로구,청운효자동,창성동,서울특별시 종로구 청운효자동,서울특별시 종로구 창성동


In [63]:
dong = data.drop(data.columns[[0, 1, 2, 3]], axis=1) # 행정동주소와 법정동주소 남기고 다 제거

In [64]:
dong.to_excel('data/행법동.xlsx', encoding='utf-8', index=False)

In [68]:
dong = pd.read_excel("data/행법동.xlsx")
print(dong.head())

             행정동주소          법정동주소
0  서울특별시 종로구 청운효자동  서울특별시 종로구 청운동
1  서울특별시 종로구 청운효자동  서울특별시 종로구 신교동
2  서울특별시 종로구 청운효자동  서울특별시 종로구 궁정동
3  서울특별시 종로구 청운효자동  서울특별시 종로구 효자동
4  서울특별시 종로구 청운효자동  서울특별시 종로구 창성동


#### 합치기 전 법정동 데이터 전처리

In [70]:
data1.replace("영등포동", "영동포동")
display(data1.head())

,fire_time,법정동주소
0,2019-01-01 00:00,강원도 삼척시 오분동
1,2019-01-01 00:00,경기도 평택시 안중읍
2,2019-01-01 00:00,서울특별시 성북구 하월곡동
3,2019-01-01 00:00,경기도 이천시 설성면
4,2019-01-01 01:00,부산광역시 부산진구 당감동


In [71]:
hb = pd.merge(data1, dong, on='법정동주소', how='left')
display(hb.head())

,fire_time,법정동주소,행정동주소
0,2019-01-01 00:00,강원도 삼척시 오분동,강원도 삼척시 남양동
1,2019-01-01 00:00,경기도 평택시 안중읍,경기도 평택시 안중읍
2,2019-01-01 00:00,서울특별시 성북구 하월곡동,서울특별시 성북구 길음제2동
3,2019-01-01 00:00,서울특별시 성북구 하월곡동,서울특별시 성북구 월곡제1동
4,2019-01-01 00:00,서울특별시 성북구 하월곡동,서울특별시 성북구 월곡제2동


In [73]:
hb.info() # 원래 fire time이 포함된 data1의 데이터가 약 4만개였는데, 조인후 총 6만개가 되었으므로 2만개가 포함이 안된것.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67196 entries, 0 to 67195
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fire_time  67196 non-null  object
 1   법정동주소      67196 non-null  object
 2   행정동주소      62680 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [74]:
hb.to_excel('data/fire_time_all.xlsx', encoding='utf-8', index=False)

#### 화재시간, 행정동, 법정동 매칭된 데이터 불러오기

In [3]:
df = pd.read_excel("data/fire_time_all_new.xlsx")
print(df.head())

          fire_time           법정동주소            행정동주소
0  2019-01-01 00:00     강원도 삼척시 오분동      강원도 삼척시 남양동
1  2019-01-01 00:00     경기도 평택시 안중읍      경기도 평택시 안중읍
2  2019-01-01 00:00  서울특별시 성북구 하월곡동  서울특별시 성북구 길음제2동
3  2019-01-01 00:00  서울특별시 성북구 하월곡동  서울특별시 성북구 월곡제1동
4  2019-01-01 00:00     경기도 이천시 설성면      경기도 이천시 설성면


In [4]:
len(df)

66143

In [9]:
df.drop_duplicates(['fire_time', '법정동주소'], keep='first', inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40032 entries, 0 to 66142
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fire_time  40031 non-null  object
 1   법정동주소      40031 non-null  object
 2   행정동주소      39416 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [10]:
df[df['행정동주소'].isnull()]

,fire_time,법정동주소,행정동주소
1651,NaN,NaN,NaN
1998,2019-01-13 23:00,경기도 성남시분당구 서현동,NaN
2308,2019-01-15 19:00,전라북도 전주시덕진구 강흥동,NaN
2377,2019-01-16 09:00,경기도 수원시장안구 송죽동,NaN
3051,2019-01-19 07:00,경기도 수원시팔달구 영동,NaN
...,...,...,...
65641,2019-12-29 14:00,경기도 수원시영통구 이의동,NaN
65734,2019-12-29 23:00,경상북도 포항시북구 대흥동,NaN
65781,2019-12-30 06:00,전라북도 전주시덕진구 호성동1가,NaN
66063,2019-12-31 16:00,세종특별자치시 세종 대평동,NaN


In [13]:
df.dropna(axis=0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39416 entries, 0 to 66142
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fire_time  39416 non-null  object
 1   법정동주소      39416 non-null  object
 2   행정동주소      39416 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [14]:
df.head()

,fire_time,법정동주소,행정동주소
0,2019-01-01 00:00,강원도 삼척시 오분동,강원도 삼척시 남양동
1,2019-01-01 00:00,경기도 평택시 안중읍,경기도 평택시 안중읍
2,2019-01-01 00:00,서울특별시 성북구 하월곡동,서울특별시 성북구 길음제2동
4,2019-01-01 00:00,경기도 이천시 설성면,경기도 이천시 설성면
5,2019-01-01 01:00,부산광역시 부산진구 당감동,부산광역시 부산진구 당감제1동


In [16]:
df = df.loc[:, ['fire_time', '행정동주소']]
df.head()

,fire_time,행정동주소
0,2019-01-01 00:00,강원도 삼척시 남양동
1,2019-01-01 00:00,경기도 평택시 안중읍
2,2019-01-01 00:00,서울특별시 성북구 길음제2동
4,2019-01-01 00:00,경기도 이천시 설성면
5,2019-01-01 01:00,부산광역시 부산진구 당감제1동


In [ ]:
df.to_excel('data/fire_time_ha.xlsx', encoding='utf-8', index=False)